In [21]:
import my_pickle as mp
import my_features as mf

# from importlib import reload

In [22]:
# reload(mp)
reload(mf)

<module 'my_features' from '/Users/gandalf/Documents/Galvanize/MatchingService/my_features.py'>

# LOAD DATA FRAMES

In [2]:
# Load User Data
user_df = mp.unjson_it('data_user')
user_df = mp.reinstate_date(user_df,['activeAt', 'available', 'birthday','created','updated'])

In [3]:
# Load Conversation Data
convo_df = mp.unjson_it('data_convo')

In [4]:
# Import Message DF
message_df = mp.unjson_it('data_message')

# MERGE USER DATA TO CONVERSATION DATA

In [8]:
# add a flag so we can identify rows with missing user data
user_df['flag'] = True

# get labels for new columns
old_names = user_df.columns
new_names = {old:old+'_sender' for old in old_names}

# merge and rename
master_df = convo_df.join(user_df, on='uid_sender') \
        .join(user_df, on='uid_receiver', rsuffix="_receiver") \
        .rename(index=str, columns = new_names)

# DROP CONVERSATIONS WITH MISSING USER DATA

In [9]:
initial_len = len(master_df)

# add False values
master_df.flag_receiver = master_df.flag_receiver.apply(lambda x: x if x==True else False)

# keep only rows with the flag
master_df = master_df[master_df.flag_receiver]

# see how many conversations we lost
print("{} conversations were removed because they were missing user data".format(initial_len-len(master_df)))

277 conversations were removed because they were missing user data


# ADD FEATURES

In [23]:
# get most of the features
master_df = mf.get_features(master_df)

columns with null values: 81


In [24]:
# add nlp stuff (should be better integrated)
text_similarity_df = mp.unjson_it('data_text_similarity')
master_df['count_similarity'] = text_similarity_df['count_similarity']
master_df['tfidf_similarity'] = text_similarity_df['tfidf_similarity']

# CONVERT TO X AND y

In [26]:
keep_X = ['I_count_receiver', 'I_count_sender',  
        'age_dif', 'age_receiver', 'age_sender', 'distance', 
        'exclaim_count_receiver','exclaim_count_sender', 'gender_receiver', 'gender_sender',
        'has_about_receiver', 'has_about_sender', 'has_facebookId_receiver','has_facebookId_sender',
        'has_linkedinId_receiver','has_linkedinId_sender', 'has_picture_receiver','has_picture_sender', 
        'has_room_receiver', 'has_room_sender','inRelationship_receiver', 'inRelationship_sender',
        'isClean_receiver', 'isClean_sender', 'isNight_receiver','isNight_sender', 
        'isStudent_receiver', 'isStudent_sender','len_about_receiver', 'len_about_sender',
        'maxCost_receiver','maxCost_sender','minCost_receiver', 'minCost_sender',
        'numRoommates_receiver','numRoommates_sender',
          'overlap_amenities', 'overlap_hobbies','overlap_neighborhoods','overlap_rent','overlap_roommate',
        'period_count_receiver', 'period_count_sender','petsOk_receiver','petsOk_sender',
        'question_count_receiver', 'question_count_sender',
        'same_city', 'same_clean',
        'same_college', 'same_country', 'same_gender', 'same_metro',
        'same_night', 'same_relate', 'same_smoking', 'same_state',
        'same_student', 'same_term', 'same_type', 'same_work',
        'sentence_count_receiver', 'sentence_count_sender',
        'smokingOk_receiver', 'smokingOk_sender', 'term_receiver','term_sender', 
        'type_receiver', 'type_sender','urgency_receiver', 'urgency_sender']
   
keep_y = ['convo_length','response']
          
drop = ['I_ratio_receiver', 'I_ratio_sender','about_receiver', 'about_sender', 
        'activeAt_receiver', 'activeAt_sender',
        'amenities_receiver', 'amenities_sender','available_receiver', 'available_sender', 
        'birthday_receiver', 'birthday_sender',
        'college_receiver', 'college_sender','created_receiver', 'created_sender',
        'exclaim_ratio_receiver','exclaim_ratio_sender',
        'facebookId_receiver', 'facebookId_sender', 'flag_receiver', 'flag_sender',
        'hobbies_receiver', 'hobbies_sender','hometownCity_receiver', 'hometownCity_sender',
        'hometownCountry_receiver', 'hometownCountry_sender','hometownState_receiver', 'hometownState_sender',
        'len_receiver','len_sender','linkedinId_receiver', 'linkedinId_sender',
        'location_receiver', 'location_sender','metro_receiver', 'metro_sender', 
        'mid_receiver','mid_sender', 'neighborhoods_receiver','neighborhoods_sender',
        'onboarded_receiver', 'onboarded_sender','period_ratio_receiver', 'period_ratio_sender', 
        'picture_receiver', 'picture_sender','question_ratio_receiver', 'question_ratio_sender',
        'sentence_ratio_receiver', 'sentence_ratio_sender','timestamp',
        'uid_receiver', 'uid_sender','updated_receiver', 'updated_sender',
        'work_receiver', 'work_sender']

In [27]:
X_df = master_df[keep_X]
X_df = X_df.fillna(X_df.mean(axis=0))
y_df = master_df[keep_y]

# PICKLE

In [28]:
mp.json_it(master_df,'data_master')
mp.json_it(X_df,'data_X')
mp.json_it(y_df,'data_y')